get the data

In [1]:
from scipy.stats import entropy
import json
import base64
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json

with open('nanoQC.html', 'r') as f:
    html_data = f.read()

# Parse the HTML content
soup = BeautifulSoup(html_data, 'html.parser')

# Find the <script> tag with the specified id
script_tag = soup.find('script', id='2293')

# Extract the content inside the <script> tag
if script_tag:
    json_content = script_tag.string.strip()
    
    # Load the JSON content
    try:
        json_data = json.loads(json_content)
        print(json_data)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
else:
    print("Script tag with id '2293' not found.")

name = list(json_data)[0]
print(name)

{'1ca820d7-5543-4709-93e8-bb4af4697efe': {'defs': [], 'roots': {'references': [{'attributes': {}, 'id': '1323', 'type': 'BasicTicker'}, {'attributes': {}, 'id': '1023', 'type': 'ResetTool'}, {'attributes': {}, 'id': '1326', 'type': 'PanTool'}, {'attributes': {}, 'id': '1003', 'type': 'DataRange1d'}, {'attributes': {'below': [{'id': '1281'}], 'center': [{'id': '1284'}, {'id': '1288'}], 'height': 400, 'left': [{'id': '1285'}], 'renderers': [{'id': '1307'}], 'title': {'id': '1394'}, 'toolbar': {'id': '1296'}, 'toolbar_location': None, 'width': 400, 'x_range': {'id': '1273'}, 'x_scale': {'id': '1277'}, 'y_range': {'id': '1275'}, 'y_scale': {'id': '1279'}}, 'id': '1272', 'subtype': 'Figure', 'type': 'Plot'}, {'attributes': {'end': 0, 'start': 100}, 'id': '1156', 'type': 'Range1d'}, {'attributes': {'axis_label': 'Mean quality score of base calls', 'coordinates': None, 'formatter': {'id': '1409'}, 'group': None, 'major_label_policy': {'id': '1410'}, 'ticker': {'id': '1323'}}, 'id': '1322', 't

get all the data indeces

In [2]:
def find_indexes_with_data(json_obj, path=[]):
    index_list = []
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            new_path = path + [key]
            if key == 'data' and 'attributes' in new_path:
                index_list.append(new_path[3])
            index_list.extend(find_indexes_with_data(value, new_path))
    elif isinstance(json_obj, list):
        for i, item in enumerate(json_obj):
            new_path = path + [i]
            index_list.extend(find_indexes_with_data(item, new_path))
    return index_list

# Assuming json_data is the loaded JSON data
index_list = find_indexes_with_data(json_data)
index_list

[62, 76, 109, 126, 129, 130, 165, 187, 199, 201, 213]

get the right indeces for the right graphs

In [3]:
graph1_indices = []
graph2_indices = []
interim_index_list = []

for index in index_list:
    data = json_data[name]['roots']['references'][index]['attributes']['data']

    if 'y' in data:
        if isinstance(data['y'], dict) and '__ndarray__' in data['y']:
            graph1_indices.append(index)
        else:
            # Check for key error for 'left'
            try:
                if json_data[name]['roots']['references'][index]['attributes']['data']['left']:
                    # If 'left' key is present, do not add to graph 1 or 2
                    pass
            except KeyError:
                interim_index_list.append(index)

    # Now go through the interim_index_list
    if index in interim_index_list:
        # If key error occurs, check if 'y' has values greater than 2
        if any(i > 2 for i in data.get('y', [])):
            # If 'y' has values greater than 2, append to graph2_indices
            pass
        else:
            # If neither criteria met, append to graph2_indices
            graph2_indices.append(index)

print("Graph 1 Indices:", graph1_indices)
print("Graph 2 Indices:", graph2_indices)

Graph 1 Indices: [109, 126, 130, 165]
Graph 2 Indices: [62, 76, 129, 199]


graph 1 for the start of the read, out put is start index to delete up to

In [4]:
# graph 1 i.e. start of read

y_1 = []
y_2 = []
y_3 = []
y_4 = []

for index in graph1_indices:
    json_string = json.dumps(json_data[name]['roots']['references'][index]['attributes']['data'], indent=2)
    data = json.loads(json_string)
    y_values = np.frombuffer(base64.b64decode(data["y"]["__ndarray__"]), dtype=data["y"]["dtype"])
    y_values = y_values.reshape(data["y"]["shape"])
    if index == graph1_indices[0]:
        y_1 = y_values
    elif index == graph1_indices[1]:
        y_2 = y_values
    elif index == graph1_indices[2]:
        y_3 = y_values
    elif index == graph1_indices[3]:
        y_4 = y_values

# store all in df, x values are the index
df_g1 = pd.DataFrame({'y_1': y_1, 'y_2': y_2, 'y_3': y_3, 'y_4': y_4})

def calculate_similarity(col1, col2, last_row):
    return abs(last_row[col1].values[0] - last_row[col2].values[0])

# Assuming df is your DataFrame created earlier
last_row_g1 = df_g1.tail(1)

# Calculate similarities
similarities = [(col, calculate_similarity('y_1', col, last_row_g1)) for col in ['y_2', 'y_3', 'y_4']]

# Find the column with the minimum absolute similarity
min_similarity_col = min(similarities, key=lambda x: x[1])[0]

# Rename columns based on minimum similarity
df_g1.rename(columns={'y_1': 'g1_A1', min_similarity_col: 'g1_A2'}, inplace=True)

# Rename other columns
for col in df_g1.columns:
    if col not in ['g1_A1', 'g1_A2']:
        df_g1.rename(columns={col: 'g1_B1'}, inplace=True)
        break

for col in df_g1.columns:
    if col not in ['g1_A1', 'g1_A2', 'g1_B1']:
        df_g1.rename(columns={col: 'g1_B2'}, inplace=True)
        break

df_g1['diff_g1A'] = df_g1.apply(lambda row: abs(row['g1_A1'] - row['g1_A2']), axis=1)

# same for y_G and y_C
df_g1['diff_g1B'] = df_g1.apply(lambda row: abs(row['g1_B1'] - row['g1_B2']), axis=1)

# find the index where dif_AT and diff_GC are < 0.01
start_index = df_g1[(df_g1['diff_g1A'] < 0.01) & (df_g1['diff_g1B'] < 0.01)].index[0]
start_index


81

graph 2 for the end of the read, output is last index of good bases (delete from)

In [5]:
# for the second graph (end of reads)
y_1 = []
y_2 = []
y_3 = []
y_4 = []

for index in graph2_indices:
    json_string = json.dumps(json_data[name]['roots']['references'][index]['attributes']['data'], indent=2)
    data = json.loads(json_string)
    y_values = data['y']
    if index == graph2_indices[0]:
        y_1 = y_values
    elif index == graph2_indices[1]:
        y_2 = y_values
    elif index == graph2_indices[2]:
        y_3 = y_values
    elif index == graph2_indices[3]:
        y_4 = y_values

# store all in df, x values are the index
df_g2 = pd.DataFrame({'y_1': y_1, 'y_2': y_2, 'y_3': y_3, 'y_4': y_4})

def calculate_similarity(col1, col2, last_row):
    return abs(last_row[col1].values[0] - last_row[col2].values[0])

# Assuming df is your DataFrame created earlier
last_row_g2 = df_g2.tail(1)

# Calculate similarities
similarities = [(col, calculate_similarity('y_1', col, last_row_g2)) for col in ['y_2', 'y_3', 'y_4']]

# Find the column with the minimum absolute similarity
min_similarity_col = min(similarities, key=lambda x: x[1])[0]

# Rename columns based on minimum similarity
df_g2.rename(columns={'y_1': 'g2_A1', min_similarity_col: 'g2_A2'}, inplace=True)

# Rename other columns
for col in df_g2.columns:
    if col not in ['g2_A1', 'g2_A2']:
        df_g2.rename(columns={col: 'g2_B1'}, inplace=True)
        break

for col in df_g2.columns:
    if col not in ['g2_A1', 'g2_A2', 'g2_B1']:
        df_g2.rename(columns={col: 'g2_B2'}, inplace=True)
        break

# compare values in columns y_A and y_T for each row and store diff_AT in a new column
df_g2['diff_A'] = df_g2.apply(lambda row: abs(row['g2_A1'] - row['g2_A2']), axis=1)

# same for y_G and y_C
df_g2['diff_B'] = df_g2.apply(lambda row: abs(row['g2_B1'] - row['g2_B2']), axis=1)

# find the index where dif_AT and diff_GC are < 0.01
end_index = df_g2[(df_g2['diff_A'] < 0.01) & (df_g2['diff_B'] < 0.01)].index[0]
end_index

# # Lowest is A or T, highest is G or C


55

In [ ]:
# print to text file
with open('nanQC_lengths.txt', 'w') as f:
    f.write(f"{start_index}\n")
    f.write(f"{end_index}\n